# Generalizations of Max-Flow

The purpose of this assignment is to learn about the min-cost flow problem, a generalization of max-flow, and to familiarize yourself with implementing and solving linear programs.

## Min-Cost Flow

Recall that a flow network with demands consists of a directed graph $G = (V, E)$, where each edge $(i,j) \in E$ has a positive integer capacity $c_{ij}$ and each node $i \in V$ has an integer demand $d_i$. In a *min-cost flow* problem, each edge $(i,j) \in E$ also has a cost (or weight) $w_{ij}$. (Note that this input generalizes the input to two important problems we discussed so far: in max flow, the edge weights were not important while in shortest paths, the edge capacities were not important.) 

Given a flow network with capacities and costs, the goal is to find a *feasible* flow $f: E \rightarrow R^+$ --that is, a flow satisfying edge capacity constraints and node demands-- that minimizes the total cost of the flow. Explicitly, the problem can be formulated as a linear program.

### Question 1

Answer Problem 1 in HW4-theoretical.

### Question 2

To implement your reduction from Problem 1 in HW4-theoretical, you will work with some standard benchmark instances for min-cost flow found [here](http://elib.zib.de/pub/Packages/mp-testdata/mincost/gte/index.html). The format of the data is described in the [Info](http://elib.zib.de/pub/Packages/mp-testdata/mincost/gte/info) file. You are to read in the graph from the data file in a form that can be solved using NetworkX's `min_cost_flow` function. Note that the data sometimes lists multiple edges between the same nodes, but with different costs or capacities. In forming the graph, you need to implement your reduction from the previous question and form a `DiGraph` instance, because the `min_cost_flow` function cannot handle multi-edges, even though the package offers `MultiDiGraph` objects.

In [1]:
import networkx as nx

def create_graph(infile):
    """Creates a directed graph as specified by the input file. Edges are annotated with 'capacity'
    and 'weight' attributes, and nodes are annotated with 'demand' attributes.
    
    Args:
        infile: the input file using the format to specify a min-cost flow problem.
        
    Returns:
        A directed graph (but not a multi-graph) with edges annotated with 'capacity' and 'weight' attributes
        and nodes annotated with 'demand' attributes.
    """
    #Load the data for original graph
    edges = dict()
    vertices = dict()
    with open(infile, 'r') as f:
        for line in f.readlines():
           
           line = line.strip().split(' ')
#           print line
           if line[0] == 'p':
#                print line
                 nb_vertices = int(line[2])
           if line[0] == 'n':
                vertices[int(line[1])] = int(line[2])    # create a dict of vertices and associate the demand with each vertex
           if line[0] == 'a':
                edge = (int(line[1]), int(line[2]))
                if edge not in edges.keys():
                    value = dict()
                    value['capacity'] = [int(line[4])]
                    value['cost'] = [float(line[5])]
                    edges[edge] = value                 # create a dict of edges and associate capacity/cost with each edge
                else: 
                    edges[edge]['capacity'].append(int(line[4]))  # if the edges already exists then 
                                                                  #add the capacity/cost  to the edge in the 
                                                                 #same key of the dict 
                    
#                    print edge , edges[edge]['capacity']        #(48, 25) [393657, 393657]
                    edges[edge]['cost'].append(float(line[5]))
#    
#    
#    #Transformation : 
                
    for edge in edges.keys():
        if len(edges[edge]['capacity']) > 1:   # If the edges has two capcities
            for k in range(len(edges[edge]['capacity'])):
                nb_vertices += 1            # add  one new vertex per exiting edge
                #Create first edge
                
                edge1 = (edge[0],nb_vertices)
                
                value = dict()
                value['capacity'] = edges[edge]['capacity'][k]
                value['cost'] = edges[edge]['cost'][k]  
                edges[edge1] = value

                #Create second edge
                edge2 = (nb_vertices, edge[1])
                value = dict()
                value['capacity'] = edges[edge]['capacity'][k]
                value['cost'] = 0 
                edges[edge2] = value

            #Remove the original edge
            del edges[edge]
#            
#    
    #Creation of the final graph 
    G = nx.DiGraph()
    for edge in edges.keys():

        G.add_edge(edge[0], edge[1])
    
    for (n1, n2) in G.edges():
        G.edge[n1][n2]['capacity'] = edges[(n1, n2)]['capacity']
        G.edge[n1][n2]['weight'] = edges[(n1, n2)]['cost']
#    
    for node1 in vertices:

        G.node[node1]['demand'] = vertices[node1]
   
    return G
#    

The following will check that your code outputs the expected min cost flow values on several test instances.

In [3]:
G_40 = create_graph('gte_bad.40')
G_6830 = create_graph('gte_bad.6830')
G_176280 = create_graph('gte_bad.176280')

print "Correct value for _40 instance:", nx.min_cost_flow_cost(G_40) == 52099553858
print "Correct value for _6830 instance:", nx.min_cost_flow_cost(G_6830) == 299390431788
print "Correct value for _176280 instance:", nx.min_cost_flow_cost(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True


## Linear Programming

Instead of using special-purpose min-cost flow solvers, you will now formulate the problems as linear programs and use general-purpose LP solvers to find the solutions.

### Question 3

Implement the following function to formulate the flow LP and return the optimal value (i.e., minimum cost over feasible flows).

In [4]:
from pulp import *

def lp_flow_value(G):
    """Computes the value of the minimum cost flow by formulating and solving
    the problem as an LP.
    
    Args:
        G: a directed graph with edges annotated with 'capacity' and 'weight'
            attrbutes, and nodes annotated with 'demand' attributes.
            
    Returns:
        The value of the minimum cost flow.
    """
    prob = LpProblem("min_cost_flow", LpMinimize)
    cost = dict()

    for edge in G.edges():
        cost[str(edge)] = G.get_edge_data(edge[0], edge[1])['weight']

    #Definition of variables
    #Flow
    f = pulp.LpVariable.dict("flow", [str(edge) for edge in G.edges()], 0)
    

    
    #Constraints

    #Capacity constraint
    for edge in G.edges():
        prob += f[str(edge)] <= G.get_edge_data(edge[0], edge[1])['capacity']
    #Demand constraint
    for node in G.nodes():
        if G.node[node] == {}:
            demand = 0
        else:
            demand = G.node[node]['demand']
            
        flow_in = []
        flow_out = []
        for edge in G.edges():
            if node == edge[1]:
                flow_in.append(f[str(edge)])
            if node == edge[0]:
                flow_out.append(f[str(edge)])
                
        prob += lpSum(flow_in) - lpSum(flow_out) == demand
                
                
    #Objective Function

    prob += lpSum([cost[str((i,j))]*f[str((i,j))] for (i,j) in G.edges()])
    status = prob.solve()
    
    return int(value(prob.objective))

The following will check that the LP finds the same optimal values as previously.

In [5]:
print "Correct value for _40 instance:", lp_flow_value(G_40) == 52099553858
print "Correct value for _6830 instance:", lp_flow_value(G_6830) == 299390431788
print "Correct value for _176280 instance:", lp_flow_value(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True
